<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Redes-neuronales" data-toc-modified-id="Redes-neuronales-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Redes neuronales</a></span><ul class="toc-item"><li><span><a href="#Entendimiento-de-los-datos" data-toc-modified-id="Entendimiento-de-los-datos-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Entendimiento de los datos</a></span></li><li><span><a href="#Pretratamiento" data-toc-modified-id="Pretratamiento-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Pretratamiento</a></span></li><li><span><a href="#Modelamiento" data-toc-modified-id="Modelamiento-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Modelamiento</a></span><ul class="toc-item"><li><span><a href="#activation" data-toc-modified-id="activation-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>activation</a></span></li><li><span><a href="#max_iter" data-toc-modified-id="max_iter-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>max_iter</a></span></li><li><span><a href="#hidden_layer_sizes" data-toc-modified-id="hidden_layer_sizes-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>hidden_layer_sizes</a></span></li><li><span><a href="#learning_rate_init" data-toc-modified-id="learning_rate_init-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>learning_rate_init</a></span></li><li><span><a href="#mejor-combinación" data-toc-modified-id="mejor-combinación-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>mejor combinación</a></span></li><li><span><a href="#DESDE-AQUÍ:-NO-EJECUTAR-DE-NUEVO-(+20-minutos)" data-toc-modified-id="DESDE-AQUÍ:-NO-EJECUTAR-DE-NUEVO-(+20-minutos)-1.3.6"><span class="toc-item-num">1.3.6&nbsp;&nbsp;</span><font color="red"><b>DESDE AQUÍ: NO EJECUTAR DE NUEVO (+20 minutos)</b></font></a></span></li><li><span><a href="#HASTA-ACÁ-(+20-minutos)" data-toc-modified-id="HASTA-ACÁ-(+20-minutos)-1.3.7"><span class="toc-item-num">1.3.7&nbsp;&nbsp;</span><font color="red"><b>HASTA ACÁ (+20 minutos)</b></font></a></span></li></ul></li></ul></li></ul></div>

# Redes neuronales

Vamos a crear un modelo de clasificación de cancer de seno con una sencilla red neuronal.

In [ ]:
import numpy as np
import pandas as pd #tratamiento de datos
import matplotlib.pyplot as plt #gráficos
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.model_selection import cross_val_score, cross_validate #método para evaluar varios particionamientos de C-V
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, LeaveOneOut #Iteradores de C-V
from sklearn.model_selection import GridSearchCV #permite buscar la mejor configuración de parámetros con C-V
from sklearn.metrics import accuracy_score, cohen_kappa_score, classification_report
from sklearn.metrics import make_scorer # permite crear una clase scorer a partir de una función de score (necesario para el kappa)
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

## Entendimiento de los datos

In [ ]:
cancer = load_breast_cancer()

In [ ]:
print(cancer['DESCR'])

Tenemos un baseline que clasifica en maligno, con 357 instancias benignas vs 212 malignas (62.7%)

In [ ]:
cancer.feature_names

In [ ]:
cancer.target_names

In [ ]:
data = pd.DataFrame(cancer.data)
data.columns = cancer.feature_names

In [ ]:
data.describe(include="all").T

Vemos que no hay missing values, y que todas las variables independientes con numéricas.
Creamos los datasets de training y de test.

In [ ]:
X = cancer['data']
y = cancer['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
df_X_train = pd.DataFrame(X_train, columns=cancer.feature_names)
df_y_train = pd.DataFrame(y_train, columns=['target'])
df_X_test = pd.DataFrame(X_test, columns=cancer.feature_names)
df_y_test = pd.DataFrame(y_test, columns=['target'])

In [ ]:
print(f"Train: {df_X_train.shape}, {df_y_train.shape}")
print(f"Test: {df_X_test.shape}, {df_y_test.shape}")

## Pretratamiento


Normalizar **no es necesario ni para las redes neuronales** desde el punto de vista que, como con la **regresión logística**, no cambia las capacidades predictivas del modelo, solo la magnitud de los coeficientes y su posible interpretación (en regresión logística, ya que en redes neuronales no es posible pensar en términos de inferencia).

Sin embargo, normalizar es una buena práctica en el sentido de que puede mejorar el número de épocas de entrenamiento necesarias, y se puede llegar a convertir en una práctica obligatoria en el caso de redes muy profundas, sobre las cuales el **gradiente** del error propagado puede **saturarse** o **desvanecerse**.

Creamos entonces un objeto escalador que aprende a transformar datos solo con respecto a los datos de entrenamiento, ya que en teoría no se conocen los de test en el momento del aprendizaje.
Se aplica luego la transformación a ambos conjuntos (train y test).

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Modelamiento

Con una regresión logística nos habría ido así:

In [ ]:
np.random.seed(1234)
logreg = LogisticRegression(random_state=1, solver='lbfgs')
logreg.fit(X_train,y_train)

In [ ]:
y_pred = logreg.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred), ", Kappa:", cohen_kappa_score(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

In [ ]:
y_pred

Con una red neuronal multi-capas obtenemos:

In [ ]:
mlp = MLPClassifier()

In [ ]:
np.random.seed(1234)
mlp = MLPClassifier(hidden_layer_sizes=(10,15,20), random_state=1, max_iter=500)
mlp.fit(X_train,y_train)

input, 1 hidden    : 30 * (10) + 10
1 hidden, 2 hidden : 10 * (15) + 15
2 hidden, 3 hidden : 15 * (20) + 20
3 hidden , output  : 20 * (1)  + 1

In [ ]:
y_pred = mlp.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred), ", Kappa:", cohen_kappa_score(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

<font color = "red">Encuentre la mejor red neuronal utilizando **GridSearchCV**, buscando la mejor combinación de los parámetros siguientes:</font>
* <font color = "red">**activation**: función de activación, a escoger entre 'logistic', 'tanh', 'relu' (valor por defecto)</font>
* <font color = "red">**max_iter**: máximo número de épocas de entrenamiento (por defecto, 200). Puede que no se necesiten todas las especificadas si se llega a convergencia).</font>
* <font color = "red">**hidden_layer_sizes**: topología de la red, vector indicando el número de neuronas por capa. Por defecto solo se tiene un capa escondidad con 100 neuronas: (100).</font>
* <font color = "red">**learning_rate_init**: tasa de aprendizaje inicial (por defecto es constante aunque se puede modificar esta tasa a medida que se va avanzando en el número de épocas). Por defecto, el valor es 0.001. </font>

In [ ]:
activation_vec = ['logistic', 'relu', 'tanh']
max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), (30, 20, 10)]
learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05]

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))

### activation

In [ ]:
import time
start = time.time() # Devuelve el tiempo actual en segundos desde el 1o de enero de 1970 (punto de referencia)

np.random.seed(1234)
parametros = {'activation': activation_vec
              }
scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1)

In [ ]:
grid.fit(X_train, y_train)

print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}%".format(
    grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))
end = time.time() # Tiempo después de finalizar el entrenamiento del modelo
print("Tiempo total: {0:.2f} minutos".format((end-start)/60))

In [ ]:
df = pd.DataFrame([(activation, acc*100, kappa*100) for (activation, acc, kappa) in
                   zip(activation_vec,
                       grid.cv_results_['mean_test_accuracy'],
                       grid.cv_results_['mean_test_kappa'],
                      )
                   ], columns = ('activation', 'Accuracy', 'Kappa'))

In [ ]:
df

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred), ", Kappa:", cohen_kappa_score(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

In [ ]:
grid.best_estimator_

### max_iter

In [ ]:
import time
start = time.time() # Devuelve el tiempo actual en segundos desde el 1o de enero de 1970 (punto de referencia)

np.random.seed(1234)
parametros = {'max_iter':max_iter_vec
              }
scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1)

In [ ]:
grid.fit(X_train, y_train)

print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
    grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))
end = time.time() # Tiempo después de finalizar el entrenamiento del modelo
print("Tiempo total: {0:.2f} minutos".format((end-start)/60))

In [ ]:
df = pd.DataFrame([(max_iter, acc*100, kappa*100) for (max_iter, acc, kappa) in
                   zip(max_iter_vec,
                       grid.cv_results_['mean_test_accuracy'],
                       grid.cv_results_['mean_test_kappa'],
                      )
                   ], columns = ('max_iter', 'Accuracy', 'Kappa'))

In [ ]:
plt.figure(figsize=(8,6))
ax = plt.gca() # get current axis
plt.plot(df.max_iter, df.Accuracy)
plt.xlabel('max_iter')
plt.ylabel('Accuracy')
plt.title('Evolución del Accuracy en función de max_iter')
plt.show()

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred), ", Kappa:", cohen_kappa_score(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

### hidden_layer_sizes

In [ ]:
import time
start = time.time() # Devuelve el tiempo actual en segundos desde el 1o de enero de 1970 (punto de referencia)

np.random.seed(1234)
parametros = {'hidden_layer_sizes':hidden_layer_sizes_vec
              }
scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1)

In [ ]:
grid.fit(X_train, y_train)

print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
    grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))
end = time.time() # Tiempo después de finalizar el entrenamiento del modelo
print("Tiempo total: {0:.2f} minutos".format((end-start)/60))

In [ ]:
df = pd.DataFrame([(hidden_layer_sizes, acc*100, kappa*100) for (hidden_layer_sizes, acc, kappa) in
                   zip(hidden_layer_sizes_vec,
                       grid.cv_results_['mean_test_accuracy'],
                       grid.cv_results_['mean_test_kappa'],
                      )
                   ], columns = ('hidden_layer_sizes', 'Accuracy', 'Kappa'))

In [ ]:
df

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred), ", Kappa:", cohen_kappa_score(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

### learning_rate_init

In [ ]:
import time
start = time.time() # Devuelve el tiempo actual en segundos desde el 1o de enero de 1970 (punto de referencia)

np.random.seed(1234)
parametros = {'learning_rate_init':learning_rate_init_vec
              }
scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1)

In [ ]:
grid.fit(X_train, y_train)

print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
    grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))
end = time.time() # Tiempo después de finalizar el entrenamiento del modelo
print("Tiempo total: {0:.2f} minutos".format((end-start)/60))

In [ ]:
df = pd.DataFrame([(learning_rate_init, acc*100, kappa*100) for (learning_rate_init, acc, kappa) in
                   zip(learning_rate_init_vec,
                       grid.cv_results_['mean_test_accuracy'],
                       grid.cv_results_['mean_test_kappa'],
                      )
                   ], columns = ('learning_rate_init', 'Accuracy', 'Kappa'))

In [ ]:
plt.figure(figsize=(8,6))
ax = plt.gca() # get current axis
plt.plot(df.learning_rate_init, df.Accuracy)
plt.xlabel('learning_rate_init')
plt.ylabel('Accuracy')
plt.title('Evolución del Accuracy en función de learning_rate_init')
plt.show()

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred), ", Kappa:", cohen_kappa_score(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

### mejor combinación

In [ ]:
activation_vec = ['logistic', 'relu', 'tanh']
max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10),
                          (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]

In [ ]:
import time
start = time.time() # Devuelve el tiempo actual en segundos desde el 1o de enero de 1970 (punto de referencia)

np.random.seed(1234)
parametros = {'activation': activation_vec,
              'max_iter':max_iter_vec,
              'hidden_layer_sizes': hidden_layer_sizes_vec,
              'learning_rate_init': learning_rate_init_vec
              }
scoring = {'kappa':make_scorer(cohen_kappa_score), 'accuracy':'accuracy'}
grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='accuracy', n_jobs=-1)

#### <font color="red"><b>DESDE AQUÍ: NO EJECUTAR DE NUEVO (+20 minutos)</b></font>

In [ ]:
grid.fit(X_train, y_train)

print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un Accuracy de {1:.2f}% y un Kappa del {2:.2f}".format(
    grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_kappa'][grid.best_index_]*100))
end = time.time() # Tiempo después de finalizar el entrenamiento del modelo
print("Tiempo total: {0:.2f} minutos".format((end-start)/60))

#### <font color="red"><b>HASTA ACÁ (+20 minutos)</b></font>

In [ ]:
df = pd.DataFrame([(acc*100, kappa*100) for (acc, kappa) in
                   zip(
                       grid.cv_results_['mean_test_accuracy'],
                       grid.cv_results_['mean_test_kappa'],
                      )
                   ], columns = ('Accuracy', 'Kappa'))

In [ ]:
df.iloc[np.argsort(-df.Accuracy),]

In [ ]:
grid.cv_results_.keys()

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred), ", Kappa:", cohen_kappa_score(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

In [ ]:
df = pd.DataFrame([(act, hidden_layers, lr, max_iter, acc*100, kappa*100) for (act, hidden_layers, lr, max_iter, acc, kappa) in
                   zip(
                       grid.cv_results_['param_activation'],
                       grid.cv_results_['param_hidden_layer_sizes'],
                       grid.cv_results_['param_learning_rate_init'],
                       grid.cv_results_['param_max_iter'],
                       grid.cv_results_['mean_test_accuracy'],
                       grid.cv_results_['mean_test_kappa'],
                      )
                   ], columns = ('Activation', 'HiddenLayers', 'LearningRate', 'MaxIter', 'Accuracy', 'Kappa'))

In [ ]:
    df.iloc[np.argsort(-df.Accuracy),].head(20)

# Búsqueda de parámetros con optimización bayesiana

La optimización de los hiper parámetros la podemos hacer de varias maneras:
- Grid search: lo que hemos hecho hasta ahora
- Random search: escoger combinaciones de valores al azar dentro de un rango determinado para cada hiperparámetro de manera independiente
- Optimización bayesiana: modelo de búsqueda "inteligente"


La **optimización bayesiana** se basa en la maximización de una métrica, dado una configuración de hiperparámetros óptima.
El paquete que vamos a utilizar es muy sencillo, y requiere la creación de una función "caja negra" que:
- recibe como argumentos los hiperparámetros del flujo de modelo a calibrar
- retorna una métrica que se quiere maximizar (e.g. el ROC AUC, el accuracy, etc.)

In [ ]:
!pip install bayesian-optimization
#conda install -c conda-forge bayesian-optimization

Vamos a crear una función de ploteo que nos permita visualizar los avances. Esta función va a recibir un diccionario de datos y va a iterar sobre él, mostrando los resultados. Esta función se podrá llamar cada iteración de la búsqueda de hiperparámetros en el proceso de optimización bayesiana.

In [ ]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

def live_plot(data_dict, figsize=(7,5), title='', win_size: int = 100):
    """
    Función para mostrar en tiempo real el progreso de la optmización bayesiana.
    """
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    for label,data in data_dict.items():
        if len(data) > win_size:
            data = data[-win_size:]
            iterations = np.arange(len(data))[-win_size:]
        else:
            iterations = np.arange(len(data))
        plt.plot(iterations, data, label=label)
    plt.title(title)
    plt.grid(True)
    plt.xlabel('Iteration')
    plt.legend(loc='center left') # the plot evolves to the right
    plt.show();

## Función de "caja negra" a optimizar

Creamos una función de "caja negra", encargada de entrenar un flujo del modelo que incluya un pretratamiento de las variables independientes (e.g. estandarización, imputación, etc.) y un modelo de clasificación con los hiperparámetros a testear.

Esta función se llamará varias veces con diferentes valores de los hiper parámetros del flujo de entrenamiento, buscando la maximización del valor que ella retorna. El proceso de esta búsqueda lo hará la librería, una vez definamos la función de caja negra y los rangos de los valores posibles de los hiper parámetros.

En este caso en particular, todas las variables son numéricas y continuas, sin embargo, para ilustrar un proceso en el que combinamos variables numéricas con categóricas, incluiremos en los flujos de entrenamiento ejemplos de  pretratamientos que deberíamos considerar en ambos casos.

La optimización bayesiana supone que todos los hiper parámetros son numéricos y contínuos, por lo que para algunos de ellos que no lo son, nos tocará ser un poco creativos. A continuación explicamos como trataremos algunos de ellos.


### Arquitectura de la red

En el caso de un flujo que incluya una red neuronal, los hiper parámetros que definen la arquitectura de la red neuronal deben tener un tratamiento muy particular. No solo tenemos que buscar el número de capas escondidas, sino el número de neuronas de ellas.

Vamos a partir de una arquitectura típica en que las primeras capas escondidas tienen mas neuronas que las siguientes. Para simplificar definimos un único hiper parámetro `model_hidden_layer_size_exp` que define toda la arquitectura. Su valor, después de convertido a entero, definirá una potencia de 2 que indicará el número de neuronas de la primera capa escondida.

Por ejemplo, si este número es 5, la primera capa escondida tendrá 2^5=32 neuronas, la siguiente capa tendrá 2^4=16 neuronas, y así sucesivamente. Establecemos un límite de 2^2=4 neuronas para la última capa escondida, teniendo en cuenta que la capa de salida en estas redes tradicionales tiene siempre una única neurona. Si el valor de este hiper parámetro es inferior a 2, se definirá una única capa de 4 neuronas.
En código establecemos entonces la arquitectura de la siguiente manera.

In [ ]:
# Este será el hiper parámetro que se definirá a través de la librería, establecemos un valor de prueba
model_hidden_layer_size_exp=2.2
max_exponent = int(model_hidden_layer_size_exp)
# Se crea una lista con los valores de las potencias de 2 de la mayor a la menor (reversada)
hidden_layer_sizes = [2**(n) for n in reversed(range(2, max_exponent+1))]
print(f"max_exponent: {max_exponent}")
print(f"arquitectura: {hidden_layer_sizes}")

In [ ]:
_ = [print(n) for n in reversed(range(2, max_exponent+1))]

### Hiper parámetros del back propagation del modelo de red neuronal tradicional

El proceso de entrenamiento por back-propagation de la red neuronal requiere la definición de los siguientes hiper parámetros:
- `model_lr_init`: learning rate a utilizar que controla la velocidad de las actualizaciones de los parámetros de la red
- `model_alpha`: controla el nivel de regularización Ridge (L2) de las neuronas
- `model_batch_size`: controla el tamaño del batch en el proceso de mini-batch gradient descent, también se define en términos de potencias de 2. Por ejemplo, un valor de 10.1 se convertirá en entero (10), y luego se tomará 2^10=1024 como tamaño del batch,
- `model_max_iter`: define el número de épocas de iteración que se aplicarán en el entrenamiento

### Imputación de datos faltantes

No hay que olvidar que estamos buscando los mejores hiper parámetros del proceso, no solo del modelo. Las redes neuronales no aceptan valores faltantes, por lo que hay que definir la estrategia de imputación.
Podemos pensar en dos maneras de imputar (pueden ser muchas mas): una que involucre el `SimpleImputer` remplazando el valor faltante por el promedio o mediana, otra que utilice el `KNNImputer` remplazando el valor faltante por el promedio o mediana de sus `K` vecinos mas cercanos.

Definimos los siguientes hiper parámetros de imputación:
- `imputer_strategy`: recibe un valor entre 0 y 1, si es inferior a 0.5 se utilizará el promedio, sino, se utilizará la mediana
- `imputer_class`: recibe un valor entre 0 y 1, si es inferior a 0.5 se utilizará el `SimpleImputer`, y para valores superiores a 0.5, se utilizará el `KNNImputer`.
- `knn_imputer_k`: en el caso de utilizar un `KNNImputer` se necesitará definir ademas otro hiper parámetro que establecerá el número de vecinos cercanos a considerar y que no tendra incidencia en el caso de `SimpleImputer`.

### Método de normalización

Similar al caso anterior en el hecho de que se refiere a un pretratamiento de datos, definimos un hiper parámetro `scaler_choice` que controlará la normalización de los datos de entrada al modelo. Si su valor es inferior a 0.5 se escala, si es superior, se normaliza entre 0 y 1.

### Tratamiento de variables categóricas

Las variables categóricas deberán ser codificadas; en este caso utilizaremos `OneHotEncoder` para tal efecto.
No definimos un hiper parámetro para este efecto, aunque podríamos considerar otros métodos de codificación como por ejemplo, **embeddings** (lo veremos en las sesiones de tratamiento de texto)

## Función de caja negra (la llamaremos `train_and_evaluate`)

Escribimos el código de la función de caja negra teniendo en cuenta lo mencionado antes.

Como lo hemos mencionado, el dataset es sencillo y no incluye variables categóricas.

In [ ]:
var_numericas = cancer.feature_names
var_categoricas = []

In [ ]:
X_train.shape

In [ ]:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from pprint import pprint
import collections

# Se crea un diccionario donde todos los valores son listas, de tal manera
# que sea fácil agregar nuevos valores de las métricas de seguimiento
# cada vez que se realice una iteración de entrenamiento, para poder ser
# graficadas con la función `live_plot`
data_plot = collections.defaultdict(list)

def add_model(data_pipeline, model) -> Pipeline:
    whole_pipeline = Pipeline([
        ("data_pipeline", data_pipeline),
        ("model", model)
    ])
    return whole_pipeline

La función `train_and_evaluate` que creamos a continuación contiene el proceso de pretratamiento de la data, de entrenamiento del flujo del modelo y de evaluación con la métrica a optimizar.

La vamos a "empacar" con una función wrapper mas adelante.
De esta manera podremos definir argumentos que no sean hiperparámetros del flujo, como lo son `verbose`, que indica si se quiere indagar mas sobre el desarrollo del proceso, y `show_live_plot`, que indica si se quiere imprimir un plot con los resultados parciales de cada iteración en tiempo real.

Además la función retorna tanto el modelo como la métrica que se desea optimizar; el proceso de optimización bayesiana del paquete que estamos utilizando requiere que solo se retorne un valor, que es lo que hará la función wrapper.

In [ ]:
def train_and_evaluate(
    #------------------------------------------
    # Hiperparámetros de tratamiento de datos
    #------------------------------------------

    # Escala de los datos: vamos a escoger entre un MinMax y una estandarización
    scaler_choice, # si <0.5: se estandariza, si >0.5: se normaliza entre 0 y 1

    # Imputación: vamos a escoger entre un SimpleScaler y un KNNImputer
    imputer_strategy, # si <0.5: se usa el promedio, si >0.5: se usa la mediana
    imputer_class,    # si <0.5: se usa SimpleImputer, si >0.5: se usa KNNImputer
    knn_imputer_k,    # hiperparámetro del KNNImputer

    #------------------------------------------
    # Hiperparámetros del modelo y de su entrenamiento
    #------------------------------------------

    # Model
    model_hidden_layer_size_exp, #controla el número de neuronas y de capas
    model_lr_init, #learning rate
    model_alpha,
    model_batch_size,
    model_max_iter,
    verbose=0,
    show_live_plot=True
) -> float: #Retorna un valor float

    #----------------------------------------------
    #--- Definición de tareas de pretratamiento ---
    #----------------------------------------------

    scaler_cls = StandardScaler if scaler_choice > 0.5 else MinMaxScaler
    imputer_strategy = "mean" if imputer_strategy > 0.5 else "median"
    if imputer_class > 0.5:
        imputer = KNNImputer(n_neighbors=int(knn_imputer_k))
    else:
        imputer = SimpleImputer(strategy=imputer_strategy)

    # Para las variables numéricas definimos un pipeline que impute
    # valores faltantes, y luego normalice
    numeric_transformer = Pipeline(
        steps=[("imputer", imputer), ("scaler", scaler_cls())]
    )

    # Para las variables categóricas definimos un column transformer
    # que traduzca las categorías a variables one hot encoded
    categorical_transformer = OneHotEncoder(handle_unknown="ignore")

    # Creamos un Column Transformer con las tareas de preprocesamiento
    # específicas a los dos tipos de variables
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, var_numericas),
            ("cat", categorical_transformer, var_categoricas),
        ]
    )

    # El pipeline se crea con un primer paso de pretratamiento. Mas adelante
    # se le agregará el paso del modelo.
    data_pipeline = Pipeline(steps=[
        ("data_processor", preprocessor),
    ])

    #---------------------------------
    #--- Configuración del modelo  ---
    #---------------------------------

    max_exponent = int(model_hidden_layer_size_exp)
    if max_exponent<2:
        max_exponent=2

    model_kwargs = dict(
        hidden_layer_sizes = [2**(n) for n in reversed(range(2, max_exponent+1))],
        batch_size=2**int(model_batch_size),
        learning_rate_init=model_lr_init,
        alpha=model_alpha,
        max_iter=int(model_max_iter),
        early_stopping=True,
        random_state=42,
    )

    if verbose:
        print("MLP Classifier params: ")
        pprint(model_kwargs)

    model = MLPClassifier(**model_kwargs)

    pipeline = add_model(data_pipeline, model)

    #------------------------------------------------
    #--- Protocolo de entrenamiento y evaluación  ---
    #------------------------------------------------

    # Seguiremos un K-fold con 3 splits aleatorios
    kf = KFold(n_splits=3, random_state=42, shuffle=True)

    # para cada fold guardamos las métricas del training y validation set
    train_fold_metrics = []
    val_fold_metrics = []

    # K-Fold cross val
    for i, (train_index, test_index) in enumerate(kf.split(df_X_train)):
        #print(f"Fold number: {i+1}")
        kX_train, kX_val = df_X_train.iloc[train_index], df_X_train.iloc[test_index]
        ky_train, ky_val = df_y_train.iloc[train_index], df_y_train.iloc[test_index]
        #print(f"Training with {kX_train.shape}")
        #print(f"Validating with {kX_val.shape}")
        pipeline.fit(kX_train, ky_train.squeeze())

        train_preds = (pipeline.predict_proba(kX_train)[:, 1]> 0.5)
        train_metric = accuracy_score(ky_train.squeeze(), train_preds)
        train_fold_metrics.append(train_metric)

        val_preds = (pipeline.predict_proba(kX_val)[:, 1]> 0.5)
        val_metric = accuracy_score(ky_val.squeeze(), val_preds)
        val_fold_metrics.append(val_metric)

    train_metric_mean = np.array(train_fold_metrics).mean()
    val_metric_mean = np.array(val_fold_metrics).mean()

    if show_live_plot:
        data_plot['train_metric'].append(train_metric_mean)
        data_plot['val_metric'].append(val_metric_mean)
        live_plot(data_plot)

    return pipeline, val_metric_mean

La función `target_func` va a "empacar" la función `train_and_evaluate`, conformándose a lo esperado por el paquete de optimización bayesiana.

In [ ]:
def target_func(**kwargs):
    model, result = train_and_evaluate(**kwargs)
    return result

Ahora que hemos creado la función de caja negra la podemos llamar directamente

In [ ]:
flujo_de_modelo, metric = train_and_evaluate(
    scaler_choice=0.3, # si <0.5: se estandariza, si >0.5: se normaliza entre 0 y 1
    imputer_strategy=0.3, # si <0.5: se usa el promedio, si >0.5: se usa la mediana
    imputer_class=0.3,    # si <0.5: se usa SimpleImputer, si >0.5: se usa KNNImputer
    knn_imputer_k=0,      # hiperparámetro del KNNImputer
    model_hidden_layer_size_exp=3, #controla el número de neuronas y de capas
    model_lr_init=0.01,
    model_alpha=0.01,
    model_batch_size=6,
    model_max_iter=50,
    verbose=1,
    show_live_plot=True
)

Podemos ver que por cada llamado, se agrega un valor a las métricas de seguimiento que vamos a plotear.

In [ ]:
data_plot

Se debe crear una instancia de **BayesianOptimization**, especificando la función a optimizar, los hiperparámetros y sus dominios de búsqueda de valores, de donde se tomarán las configuraciones a evaluar.

In [ ]:
from bayes_opt import BayesianOptimization

Definimos los intervalos sobre los cuáles se va a realizar la búsqueda para cada hiper parámetro

In [ ]:
pbounds = {'scaler_choice': (0, 1), 'imputer_strategy': (0, 1), 'imputer_class': (0, 1), 'knn_imputer_k': (1,16),
           'model_hidden_layer_size_exp': (2, 5), 'model_lr_init': (0.005, 0.5), 'model_alpha': (0.001, 1),
           'model_batch_size':(3, 7), 'model_max_iter':(50,50)}

Creamos el optimizador bayesiano, especificando la función caja negra a maximizar y la configuración de hiper parámetros

In [ ]:
data_plot = collections.defaultdict(list)

In [ ]:
optimizer = BayesianOptimization(
    f=target_func,
    pbounds=pbounds,
    random_state=42,
    verbose=2,
)

Si se quiere empezar por una configuración en particular, se pueden programar antes de lanzar la búsqueda automática:

In [ ]:
optimizer.probe(
    params={'scaler_choice': 0.3, 'imputer_strategy': 0.3, 'imputer_class': 0.3, 'knn_imputer_k': 1,
            'model_hidden_layer_size_exp': 3, 'model_lr_init': 0.01, 'model_alpha': 0.01, 'model_batch_size': 6, 'model_max_iter': 50}, lazy=True)
optimizer.probe(
    params={'scaler_choice': 0.3, 'imputer_strategy': 0.3, 'imputer_class': 0.3, 'knn_imputer_k': 1,
            'model_hidden_layer_size_exp': 2, 'model_lr_init': 0.1, 'model_alpha': 0.01, 'model_batch_size': 6, 'model_max_iter': 50}, lazy=True)
optimizer.probe(
    params={'scaler_choice': 0.3, 'imputer_strategy': 0.3, 'imputer_class': 0.3, 'knn_imputer_k': 1,
            'model_hidden_layer_size_exp': 4, 'model_lr_init': 0.05, 'model_alpha': 0.01, 'model_batch_size': 6, 'model_max_iter': 50}, lazy=True)

Se puede utilizar directamente el objeto de **BayesianOptimization**, usando su método **maximize** que optimiza la función definida. Entre los parámetros de este método los más importante son:

- init_points: el número de pasos de exploración aleatorios a ejecutar para incializar el proceso gausiano.
- n_iter: el número de iteraciones del proceso gausiano de búsqueda de configuraciones de hiperparámetros. A mayor número de pasos, mejor la maximización, pero mas largo el proceso.

In [ ]:
%%time
optimizer.maximize(
    init_points=2,
    n_iter=30,
)

Se puede retomar la búsqueda si se desea

In [ ]:
%%time
optimizer.maximize(
    n_iter=10,
)

Se obtienen luego los hiperparámetros óptimos:

In [ ]:
optimizer.max

Obtenemos el mejor modelo

In [ ]:
best_model, best_result = train_and_evaluate(**optimizer.max["params"], show_live_plot=False)

In [ ]:
best_result